In [ ]:
import pandas as pd
import requests
import locale
from datetime import datetime, timedelta

In [ ]:
UNAME = '' #fill ur uname account
PASSWORD = '' #fill ur password account
ACTIVITY_ID = 0000 #fill ur activity id

In [ ]:
try:
  url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
  obj = {
      'email': UNAME,
      'password': PASSWORD
  }

  response = requests.post(url, json=obj)
  token = response.json()['data'].get('access_token')

  headers = dict()
  headers["Accept"] = "application/json"
  headers["Authorization"] = f'Bearer {token}'

  url =  "https://api.kampusmerdeka.kemdikbud.go.id/mbkm/mahasiswa/active-activities/my"
  response = requests.get(url, headers=headers)

  url = f'https://api.kampusmerdeka.kemdikbud.go.id/studi/report/allweeks/{ACTIVITY_ID}'
  response = requests.get(url, headers=headers)
  reports = response.json().get('data')
  print('success fetch')
except Exception as e :
  print(e)

In [ ]:
weekly_reports = []
daily_reports = []

for report in reversed(list(reports)):
    date = report.get('start_date')[0:10]
    date =  datetime.strptime(date, '%Y-%m-%d')

    date_format = date.strftime("%d %B %Y")
    date_format = date_format.split()

    weekly_reports.append([report.get("learned_weekly"), f"{date_format[1]} {date_format[2]}"])

    for j, daily_report in enumerate(report.get('daily_report')):
        if j == 0:
            dt = date 
        else:
            dt = dt + timedelta(days=1)

        d = dt.strftime("%A, %d %B %Y")
        daily_reports.append([daily_report.get('report'), d])

In [ ]:
df_weekly = pd.DataFrame(weekly_reports, columns=['Deskripsi', 'Bulan'])
df_weekly.sample(5)

,Deskripsi,Bulan
9,Minggu ini saya mempelajari banyak hal.. mulai...,April 2022
15,Melakukan sesi mentoring bersama instruktur da...,June 2022
3,Menyelesaikan seluruh course Google IT Automat...,March 2022
13,Melakukan webscrapping terhadap beberapa kompe...,May 2022
6,Minggu ini belajar banyak hal yaitu Materi mac...,April 2022


In [ ]:
df_daily = pd.DataFrame(daily_reports, columns=['Deskripsi', 'Hari'])
df_daily.sample(5)

,Deskripsi,Hari
13,Mengikuti kelas english session dengan pemater...,"Thursday, 10 March 2022"
49,Memahami implementasi data pipeline dengan ten...,"Friday, 29 April 2022"
59,Melakukan data preprocessing dataset. Dikarena...,"Friday, 13 May 2022"
69,Mengikuti kegiatan weekly consultation yang me...,"Friday, 27 May 2022"
90,Tidak ada operasional,"Monday, 27 June 2022"


In [ ]:
df_weekly.to_csv('Weekly Report.csv', index=False)
df_daily.to_csv('Daily Report.csv', index=False)